# **Wprowadzenie do PyTorch**
PyTorch to elastyczna i wydajna biblioteka do uczenia maszynowego i głębokiego uczenia. Jest szeroko stosowana dzięki swojej prostocie, wsparciu dynamicznych grafów obliczeniowych oraz obsłudze obliczeń na GPU.

## **1. Instalacja i konfiguracja**
PyTorch można zainstalować za pomocą pip lub conda:
```bash
pip install torch torchvision torchaudio
```
Sprawdzenie wersji PyTorch:

In [1]:
import torch
print(torch.__version__)

2.5.1+cpu


## **2. Tensory w PyTorch**
Tensory to podstawowa jednostka danych w PyTorch, podobna do tablic NumPy, ale z obsługą GPU.

In [2]:
# Tworzenie tensora
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
print("Tensor x:", x)

# Operacje matematyczne
y = torch.tensor([[5, 6], [7, 8]])
z = x + y
print("Dodawanie tensorów:", z)

# Przeniesienie na GPU (jeśli dostępne)
device = "cuda" if torch.cuda.is_available() else "cpu"
x_gpu = x.to(device)
print("Tensor na GPU:", x_gpu)

Tensor x: tensor([[1., 2.],
        [3., 4.]])
Dodawanie tensorów: tensor([[ 6.,  8.],
        [10., 12.]])
Tensor na GPU: tensor([[1., 2.],
        [3., 4.]])


## **3. Przykład z danymi MNIST**
### **a) Przygotowanie danych**
Korzystamy z TorchVision, aby załadować dane MNIST.

In [3]:
from torchvision import datasets, transforms

# Transformacje danych
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Ładowanie danych
train_dataset = datasets.MNIST(root='mnist_data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='mnist_data', train=False, transform=transform, download=True)

# DataLoader (do iteracji po danych)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to mnist_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to mnist_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz to mnist_data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



### **b) Budowa modelu**
Definiujemy prostą sieć neuronową z jedną warstwą ukrytą.

In [4]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Model dla MNIST
model = NeuralNet(input_size=28*28, hidden_size=128, num_classes=10)

### **c) Trenowanie modelu**
Używamy CrossEntropyLoss jako funkcji strat i optymalizatora Adam.

In [5]:
import torch.optim as optim

# Funkcja kosztu
criterion = nn.CrossEntropyLoss()

# Optymalizator
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pętla trenowania
for epoch in range(5):  # Liczba epok
    for images, labels in train_loader:
        # Przekształcenie obrazów (28x28 -> 784)
        images = images.view(-1, 28*28)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass i optymalizacja
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.3032
Epoch [2/5], Loss: 0.3109
Epoch [3/5], Loss: 0.2280
Epoch [4/5], Loss: 0.0656
Epoch [5/5], Loss: 0.0244


### **d) Ocena modelu**
Oceniamy dokładność modelu na zbiorze testowym.

In [6]:
# Ocena dokładności na zbiorze testowym
model.eval()  # Przełączenie na tryb ewaluacji
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Dokładność: {100 * correct / total:.2f}%")

Dokładność: 96.95%
